<a href="https://colab.research.google.com/github/YunshuoTian/nlp/blob/main/vector_search_airbnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 12.6 MB/s eta 0:00:00


In [3]:
import pymongo
import requests
from google.colab import userdata
import time

mongo_secret = userdata.get('mongo_secret')
client = pymongo.MongoClient(mongo_secret)
hf_token = userdata.get('hf_read')

In [ ]:
db = client.sample_airbnb
collection = db.listingsAndReviews
collection.find_one({'summary':{"$exists": True}, 'summary_embedding': {'$exists': False}})

In [8]:
embedding_url = "https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/all-MiniLM-L6-v2"

def generate_embedding(text: str) -> list[float]:

    response = requests.post(
        embedding_url,
        headers={"Authorization": f"Bearer {hf_token}"},
        json={"inputs": text})

    if response.status_code == 503:
        time.sleep(20)
    if response.status_code != 200:
        raise ValueError(f"Request failed with status code {response.status_code}: {response.text}")

    return response.json()

In [9]:
for doc in collection.find({'summary':{"$exists": True}, 'summary_embedding': {'$exists': False}}).limit(100):
	doc['summary_embedding'] = generate_embedding(doc['summary'])
	collection.replace_one({'_id': doc['_id']}, doc)

In [ ]:
collection.count_documents({'summary_embedding': {'$exists': True}})

In [15]:
query = "apartment in city center of new york"

results = collection.aggregate([
  {"$vectorSearch": {
    "queryVector": generate_embedding(query),
    "path": "summary_embedding",
    "numCandidates": 100,
    "limit": 4,
    "index": "summary_index",
      }}
])

In [16]:
for document in results:
    print(f'Name: {document["name"]},\nSummary: {document["summary"]},\nSpace: {document["space"]}\n')

Name: Private Cozy Bedroom in Brooklyn,
Summary: Your private room is located in our family's lovely 3-bedroom condo, a welcoming space for you to call home while enjoying all that New York's five boroughs have to offer. We are artists and students of culture and will be happy to help with recommendations during your stay. We also have another bedroom listing with more space: https://www.airbnb.com/rooms/135393?s=8&user_id=663764&ref_device_id=834476da76042c7d3d85486a5eaac86e8cc4de3b,
Space: To yourself, you'll have a cozy room furnished with a comfortable and adjustable Ikea bed that pulls out from a twin to become a double. Your room also has a small refrigerator/freezer. yay! Clean sheets, blankets, pillows, and shampoo/conditioner are provided, as well as fresh towels and some closet space. (There are no additional charges for these items.) (Note: When set-up as a double, the bed will take up most of the space in the room, though you may push it back into a twin for more space duri